In [1]:
import yfinance as yf
import seaborn as sns
import pandas  as pd
import numpy as np
import pandas_market_calendars as mcal
import pytz

from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from FMNM.BS_pricer import BS_pricer
from FMNM.Parameters import Option_param
from FMNM.Processes import Diffusion_process

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

from utils.gbm import GBMAnalyzer
from utils.env import OptionTradingEnv
from utils.dataFetcher import OptionsDataFetcher as odf

In [2]:
from stable_baselines3 import PPO

env = OptionTradingEnv('AAPL', portfolio={}, end_date_str='2024-12-31', yrs=1)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)

obs, _ = env.reset()
for _ in range(100):
    action, _states = model.predict(obs)
    obs, rewards, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
        obs, _ = env.reset()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 230      |
|    ep_rew_mean     | 0.211    |
| time/              |          |
|    fps             | 361      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 230       |
|    ep_rew_mean          | 1.29      |
| time/                   |           |
|    fps                  | 313       |
|    iterations           | 2         |
|    time_elapsed         | 13        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0171557 |
|    clip_fraction        | 0.378     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.59     |
|    e

In [3]:
env.stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,log_return,hist_vol
Date,,,,,,,,,
2024-01-31 00:00:00-05:00,185.680224,185.739801,183.009793,183.059418,55467800,0.0,0.0,-0.019547,0.211112
2024-02-01 00:00:00-05:00,182.652411,185.590883,182.483648,185.501541,64885400,0.0,0.0,0.013252,0.214585
2024-02-02 00:00:00-05:00,178.552428,185.968122,177.946862,184.498886,102518000,0.0,0.0,-0.005420,0.209992
2024-02-05 00:00:00-05:00,186.782151,187.874160,184.488947,186.315567,69668800,0.0,0.0,0.009798,0.211272
2024-02-06 00:00:00-05:00,185.501564,187.933750,185.412222,187.923828,43490800,0.0,0.0,0.008595,0.196488
...,...,...,...,...,...,...,...,...,...
2024-12-23 00:00:00-05:00,254.156919,255.034791,252.840088,254.655716,40858800,0.0,0.0,0.003060,0.141182
2024-12-24 00:00:00-05:00,254.875189,257.588630,254.675658,257.578674,23234700,0.0,0.0,0.011413,0.140158
2024-12-26 00:00:00-05:00,257.568678,259.474086,257.010028,258.396667,27237100,0.0,0.0,0.003171,0.139420


In [4]:
env = OptionTradingEnv("TSLA", 10_000, "2025-01-01", 1)

In [5]:
env.options_data

,contractSymbol,lastTradeDate,strike,lastOptionPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,type,expiration
0,TSLA250606C00050000,2025-05-30 18:14:48+00:00,50.0,300.25,295.05,296.75,-8.149994,-2.642670,5.0,5.0,6.953126,True,REGULAR,USD,call,2025-06-06
1,TSLA250606C00055000,2025-05-23 18:52:06+00:00,55.0,286.45,290.05,291.75,0.000000,0.000000,4.0,4.0,6.609377,True,REGULAR,USD,call,2025-06-06
2,TSLA250606C00060000,2025-05-29 19:24:24+00:00,60.0,298.60,285.05,286.75,0.000000,0.000000,2.0,8.0,6.292971,True,REGULAR,USD,call,2025-06-06
3,TSLA250606C00065000,2025-05-29 19:24:24+00:00,65.0,293.60,280.05,281.75,0.000000,0.000000,2.0,10.0,6.007815,True,REGULAR,USD,call,2025-06-06
4,TSLA250606C00070000,2025-05-29 19:57:49+00:00,70.0,288.72,275.05,276.75,0.000000,0.000000,5.0,4.0,5.742190,True,REGULAR,USD,call,2025-06-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4275,TSLA271217P00650000,2025-05-30 15:39:38+00:00,650.0,332.35,339.60,341.20,0.300018,0.090353,18.0,0.0,0.467702,True,REGULAR,USD,put,2027-12-17
4276,TSLA271217P00660000,2025-05-29 17:14:25+00:00,660.0,339.35,346.40,350.55,0.000000,0.000000,2.0,2.0,0.470014,True,REGULAR,USD,put,2027-12-17
4277,TSLA271217P00670000,2025-05-29 17:12:41+00:00,670.0,348.01,355.00,359.10,0.000000,0.000000,4.0,3.0,0.468053,True,REGULAR,USD,put,2027-12-17
4278,TSLA271217P00690000,2025-05-23 14:00:49+00:00,690.0,380.54,372.10,376.30,0.000000,0.000000,3.0,3.0,0.463949,True,REGULAR,USD,put,2027-12-17
